In [1]:
!pip install pycountry

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 50.9 MB/s eta 0:00:00


In [2]:
import pycountry
import pandas as pd
import numpy as np

data = pd.read_csv("/kaggle/input/united-nations-general-assembly-voting-dataset/UNGA Voting Data Mar 31 2025.csv")

/tmp/ipykernel_13/2433126052.py:5: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("/kaggle/input/united-nations-general-assembly-voting-dataset/UNGA Voting Data Mar 31 2025.csv")


In [3]:
data = data[data['ms_vote']!='X']
data = data.drop(columns=['undl_id', 'ms_name', 'date', 'session', 'draft', 'committee_report', 'meeting', 'title', 'agenda_title', 'subjects', 'total_yes', 'total_no', 'total_abstentions', 'total_non_voting', 'total_ms', 'undl_link'])

code_to_country = {country.alpha_3: country.name for country in pycountry.countries}

data['ms_code'] = data['ms_code'].map(code_to_country)
data = data.rename(columns={'ms_code':'country'})

data = data[~data['country'].isin([
    'Holy See (Vatican City State)', 'Congo', 'Czechia', 'Congo, The Democratic Republic of'
])]

data['country'] = data['country'].replace({
    'Viet Nam': 'Vietnam',
    'Brunei Darussalam': 'Brunei',
    'Iran, Islamic Republic of': 'Iran',
    "Lao People's Democratic Republic": 'Laos',
    'Türkiye': 'Turkey',
    'Syrian Arab Republic': 'Syria',
    'Russian Federation': 'Russia',
    'Palestine, State of': 'Palestine',
    'Korea, Republic of': 'South Korea',
    "Korea, Democratic People's Republic of": 'North Korea',
    'Bolivia, Plurinational State of': 'Bolivia',
    'Moldova, Republic of': 'Moldova',
    'Tanzania, United Republic of': 'Tanzania',
    'Micronesia, Federated States of': 'Micronesia',
    'Venezuela, Bolivarian Republic of': 'Venezuela'
})

In [4]:
grouped = data.groupby(['resolution', 'ms_vote'], dropna=True)['country'].agg(list)
grouped_str = grouped.apply(lambda x: ';'.join(map(str, x)))
df = grouped_str.reset_index(name='countries')
df = df.drop(columns=['ms_vote'])

In [5]:
df.to_csv('vote_groupings.csv', index=False)